In [8]:
def read_query_as_df(query):
    connection = None
    connection = psycopg2.connect(user="mjjyypvqfcescn", password="e93dc8ef167aa960b56248e5a2231cbc7d7ad5854266e7df2ab867763f065629", host="ec2-63-34-97-163.eu-west-1.compute.amazonaws.com",port="5432",database="d94t9tih4i30sp")
    cursor = connection.cursor()  
    temp = []
    result = None
    cursor.execute(query)
    results = cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]
    connection.close()
    for result in results:
        result = list(result)
        temp.append(result)
    df = pd.DataFrame(temp, columns=colnames)
    connection.close()
    return df

In [97]:
import psycopg2
import psycopg2.extras
import pandas as pd
panel_type = 1
query = """SELECT ppmr.*, pm.material 
FROM production.panel_material_req ppmr
JOIN production.material pm
ON pm.id = ppmr.material_id
where ppmr.panel_type_id = %s"""%(panel_type)
material_req = read_query_as_df(query)
display(material_req)

,id,material_id,panel_type_id,amount,material
0,7,4,1,5.20,Aluminum Frame
1,8,7,1,1.00,Junction Box
2,6,5,1,0.30,Tabbing Ribbon
3,5,6,1,0.13,Buss Ribbon
4,3,1,1,1.00,Solar Glass
5,1,2,1,60.00,Solar Cell
6,2,9,1,1.60,EVA
7,4,8,1,1.60,Tedlar


In [98]:
query = """SELECT ppmr.material_id, sum(ppmr.amount) FROM production.panel_material_req ppmr
right join production.solar_panel psp
on psp.panel_type_id = ppmr.panel_type_id
group by ppmr.material_id"""
material_used = read_query_as_df(query)
display(material_used)

,material_id,sum
0,8,83.20
1,9,83.20
2,7,52.00
3,1,52.00
4,5,15.60
5,2,3120.00
6,4,270.40
7,6,6.76


In [100]:
query = """SELECT pqli.material_id, pqli.quantity, pqli.unit_price, qp.quote_date
FROM finance.quote_line_items pqli
JOIN finance.quote qp
ON pqli.quote_id = qp.id
right JOIN finance.purchase_orders fpo
ON fpo.quote_id = qp.id
WHERE material_id in (SELECT ppmr.material_id FROM production.panel_material_req ppmr
where ppmr.panel_type_id = 1)"""
material_ordered = read_query_as_df(query)
display(material_ordered.sort_values(by=['quote_date']))


,material_id,quantity,unit_price,quote_date
23,2,100.00,1.25000,2018-04-23
24,2,100.00,0.98000,2018-04-23
0,2,50.00,0.95000,2018-05-01
22,2,50.00,0.90000,2018-05-01
3,1,5.00,8.74840,2018-09-22
4,4,26.32,2.15800,2018-09-22
21,2,310.00,1.10000,2018-09-22
7,5,2.00,18.03000,2018-09-22
18,9,30.00,1.93000,2018-09-22
16,8,15.00,3.09000,2018-09-22


In [104]:
total = 0
for index, row in material_req.iterrows():
    print(row["material"], row["material_id"])
    amount_used = material_used[material_used["material_id"] == row["material_id"]]["sum"].values
    print("used so far: ", amount_used[0])
    material_ordered_filtered_sorted = material_ordered[material_ordered["material_id"]== row["material_id"]].sort_values(by=['quote_date'])
    display(material_ordered_filtered_sorted[material_ordered_filtered_sorted["quantity"].cumsum() > amount_used[0]])
    unit_price = material_ordered_filtered_sorted[material_ordered_filtered_sorted["quantity"].cumsum() > amount_used[0]]["unit_price"].head(1).values
    print("Unit price: ", unit_price[0])
    print(row["material"],":", unit_price[0]*row["amount"])
    total += unit_price[0]*row["amount"]
    print(total)

Aluminum Frame 4
used so far:  270.39999999999975


,material_id,quantity,unit_price,quote_date
5,4,1210.72,1.45327,2019-09-30


Unit price:  1.45327
Aluminum Frame : 7.557004000000001
7.557004000000001
Junction Box 7
used so far:  52.0


,material_id,quantity,unit_price,quote_date
12,7,230.0,3.52,2019-09-30
13,7,1000.0,3.20,2021-04-05


Unit price:  3.52
Junction Box : 3.52
11.077004
Tabbing Ribbon 5
used so far:  15.600000000000016


...

Unit price:  0.559
Solar Cell : 33.540000000000006
57.32435400000001
EVA 9
used so far:  83.19999999999996


,material_id,quantity,unit_price,quote_date
20,9,1000.0,1.11,2019-09-30
19,9,3600.0,1.65,2021-04-05


Unit price:  1.11
EVA : 1.7760000000000002
59.10035400000001
Tedlar 8
used so far:  83.19999999999996


,material_id,quantity,unit_price,quote_date
15,8,500.0,1.76,2019-09-30
17,8,1800.0,1.52,2021-04-05


Unit price:  1.76
Tedlar : 2.8160000000000003
61.91635400000001


In [102]:
material_used
unit_price = material_ordered_filtered_sorted[material_ordered_filtered_sorted["quantity"].cumsum() > amount[0]]["unit_price"].tail(1).values
print(unit_price[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [46]:
amount = material_used[material_used["material_id"] == 1]["sum"].values
print(amount[0])

52.0
